**C4.5算法**

C4.5算法是基于ID3算法的改进，C4.5在生成的过程中使用信息增益比来选择特征。

输入:训练数据集$D$，特征集$A$阈值$\varepsilon$;  
输出:决策树$T$.

第一步：若$D$中所有实例属于同一类$C_k$,则置$T$为单结点树，并将类$C_k$作为该节点的类标记，返回$T$；

第二步：若$A=\emptyset$,则$T$为单节点树，并将$D$中实例数最大的类$C_k$作为该结点的类标记，返回$T$； 

第三步：否则，按信息增益算比式子计算$A$中各特征对$D$的信息增益比，选择信息增益比最大的特征$A_g$； 

第四步：如果$A_g$的信息增益比小于阈值$\varepsilon$，则置$T$为单结树，并将$D$中实例数最大的类$C_k$作为该结点的类标记，返回$T$；  

第五步：否则，对$A_g$的每一个可能值$a_i$,依$A_g=a_i$将$D$分割为若干非空子集$D_i$,将$D_i$中实例数最大的类作为标记，构建子结点，由结点及其子结点构成树$T$,返回$T$；

第六步：对第$i$个子结点，以$D_i$为训练集，以$A-\{A_g\}$为特征集，递归地调用步骤一至五，得到子树$T_i$,返回$T_i$。


**书中例5.3  利用C4.5算法生成决策树 python代码 **

In [1]:
# C4.5算法
def recurse_train_C45(train_set,train_label,features):
    
    LEAF = 'leaf'
    INTERNAL = 'internal'

    # 步骤1—如果训练集train_set中的所有实例都属于同一类Ck
    label_set = set(train_label)
    if len(label_set) == 1:
        return Tree(LEAF,Class = label_set.pop())

    # 步骤2—如果特征集features为空
    class_len = [(i,len(list(filter(lambda x:x==i,train_label)))) for i in range(class_num)] # 计算每一个类出现的个数
    (max_class,max_len) = max(class_len,key = lambda x:x[1])
    
    if len(features) == 0:
        return Tree(LEAF,Class = max_class)

    # 步骤3—计算信息增益,并选择信息增益最大的特征
    max_feature = 0
    max_gda = 0
    D = train_label
    for feature in features:
        # print(type(train_set))
        A = np.array(train_set[:,feature].flat) # 选择训练集中的第feature列（即第feature个特征）
        gda = calc_ent_grap(A,D)
        if calc_ent(A) != 0:  # 计算信息增益比，这是与ID3算法唯一的不同
            gda /= calc_ent(A)
        if gda > max_gda:
            max_gda,max_feature = gda,feature

    # 步骤4—信息增益小于阈值
    if max_gda < epsilon:
        return Tree(LEAF,Class = max_class)

    # 步骤5—构建非空子集
    sub_features = list(filter(lambda x:x!=max_feature,features))
    tree = Tree(INTERNAL,feature=max_feature)

    max_feature_col = np.array(train_set[:,max_feature].flat)
     # 保存信息增益最大的特征可能的取值 (shape[0]表示计算行数)
    feature_value_list = set([max_feature_col[i] for i in range(max_feature_col.shape[0])])
    for feature_value in feature_value_list:

        index = []
        for i in range(len(train_label)):
            if train_set[i][max_feature] == feature_value:
                index.append(i)

        sub_train_set = train_set[index]
        sub_train_label = train_label[index]

        sub_tree = recurse_train_C45(sub_train_set,sub_train_label,sub_features)
        tree.add_tree(feature_value,sub_tree)

    return tree

#### 定义经验熵、条件熵、信息增益等

In [2]:
# 计算数据集x的经验熵H(x)
def calc_ent(x):
    x_value_list = set([x[i] for i in range(x.shape[0])])
    ent = 0.0
    for x_value in x_value_list:
        p = float(x[x == x_value].shape[0]) / x.shape[0]
        logp = np.log2(p)
        ent -= p * logp

    return ent

# 计算条件熵H(y/x)
def calc_condition_ent(x, y):
    x_value_list = set([x[i] for i in range(x.shape[0])])
    ent = 0.0
    for x_value in x_value_list:
        sub_y = y[x == x_value]
        temp_ent = calc_ent(sub_y)
        ent += (float(sub_y.shape[0]) / y.shape[0]) * temp_ent

    return ent

# 计算信息增益
def calc_ent_grap(x,y):
    base_ent = calc_ent(y)
    condition_ent = calc_condition_ent(x, y)
    ent_grap = base_ent - condition_ent

    return ent_grap

#### 定义树结构

In [3]:
class Tree(object):
    def __init__(self,node_type,Class = None, feature = None):
        self.node_type = node_type  # 节点类型（internal或leaf）
        self.dict = {} # dict的键表示特征Ag的可能值ai，值表示根据ai得到的子树 
        self.Class = Class  # 叶节点表示的类，若是内部节点则为none
        self.feature = feature # 表示当前的树即将由第feature个特征划分（即第feature特征是使得当前树中信息增益最大的特征）

    def add_tree(self,key,tree):
        self.dict[key] = tree

    def predict(self,features): 
        if self.node_type == 'leaf' or (features[self.feature] not in self.dict):
            return self.Class

        tree = self.dict.get(features[self.feature])
        return tree.predict(features)

#### 图像处理需要用到二值化

In [4]:
# 二值化
def binaryzation(img):
    cv_img = img.astype(np.uint8)
    cv2.threshold(cv_img,50,1,cv2.THRESH_BINARY_INV,cv_img)
    return cv_img

def binaryzation_features(trainset):
    features = []

    for img in trainset:
        img = np.reshape(img,(28,28))
        cv_img = img.astype(np.uint8)

        img_b = binaryzation(cv_img)
        # hog_feature = np.transpose(hog_feature)
        features.append(img_b)

    features = np.array(features)
    features = np.reshape(features,(-1,feature_len))

    return features

In [5]:
def train(train_set,train_label,features):
    return recurse_train_C45(train_set,train_label,features)

def predict(test_set,tree):
    result = []
    for features in test_set:
        tmp_predict = tree.predict(features)
        result.append(tmp_predict)
    return np.array(result)

## 训练测试

In [6]:
import cv2
import time
import numpy as np
import pandas as pd

from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score

E:\softinstall\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
class_num = 10 
feature_len = 784 
epsilon = 0.001  # 设定阈值

print("读取数据")


raw_data = pd.read_csv('../data/train.csv', header=0)  # 读取csv数据
data = raw_data.values
    
imgs = data[::, 1::]
features = binaryzation_features(imgs) # 图片二值化(很重要，不然预测准确率很低)
labels = data[::, 0]

# 避免过拟合，采用交叉验证，随机选取33%数据作为测试集，剩余为训练集
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size=0.33, random_state=0)


# 通过C4.5算法生成决策树
print('开始训练')
time1 = time.time()
tree = train(train_features,train_labels,list(range(feature_len)))
time2 = time.time()
print('训练花费时间： %f seconds' % (time2 - time1))

print('开始预测')
test_predict = predict(test_features,tree)
time3 = time.time()
print('预测花费时间 %f seconds' % (time3 - time2))
    
# 预测精度
for i in range(len(test_predict)):
    if test_predict[i] == None:
        test_predict[i] = epsilon
score = accuracy_score(test_labels, test_predict)
print("精度为： %f" % score)

读取数据
开始训练
训练花费时间： 1861.676888 seconds
开始预测
预测花费时间 0.972255 seconds
精度为： 0.842424


**上述案例均已通过**

----
参考代码：https://github.com/fuqiuai/lihang_algorithms

整理制作：深度学习学研社

<div>
<table align="left" border="1" bordercolor="#000000">
    <div>
    <tr>
        <td>
            微信公众号：ID: AI_class_vip<br>
            <img src="../image/gongzhonghao.jpg" width="150" height="150" align="left"/>    
        </td>
    </tr>
    </div>
    <div>
    <tr>
        <td>
        知识星球：机器学习交流学习圈：<br>
    <img src="../image/dlzhishixingqiu.jpg" width="150" height="150" align="left"/>  
        </td>
    </tr>
        </div>
    <div>
     <tr>
        <td>
        配置环境：python 3.4+  
        </td>
    </tr>
        </div>
</table>
</div>
